In [1]:
import torch 

torch.set_default_device("cuda")

In [2]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import sklearn

Обрабатываем SSTS файлы:

In [4]:
from docx2python import docx2python
import re

def parse_docx_text_SSTS(file_path):
    docx_content = docx2python(file_path)
    text_content = docx_content.text

    sections = {}
    section_index = None
    current_section = []
    pre_section_content = []

    for line in text_content.split('\n'):
        line = line.strip()
        if not line:
            continue

        pattern = r'\((?:[^()]*[&|][^()]*)+\)'
        pattern2 = r'\((?:[a-zA-Z](?:[|&][a-zA-Z])*)\)'
        match = re.search(pattern2, line)
        if match:
            if current_section and section_index is not None:
                sections[section_index] = ' '.join(current_section)
            elif not section_index:
                # If no section has been started yet, store pre-section content
                sections["pre_section"] = ' '.join(pre_section_content)
            section_index = line  # Use the entire line as the section index
            current_section = [line]
        else:
            if section_index is None:
                pre_section_content.append(line)
            else:
                current_section.append(line)

    if current_section and section_index is not None:
        sections[f"{section_index}."] = ' '.join(current_section)

    return sections


Заменяем алгебру логики: 


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" # the device to load the model onto

model_name = "Qwen/Qwen2.5-14B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda",
    cache_dir="/home/dev/llm_weights"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
def generate(prompt, system_prompt=None, max_new_tokens=1):
    messages = [
        {"role": "system", "content": "You are a helpful assistant." if system_prompt is None else system_prompt},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=max_new_tokens,
        temperature=0.00000001,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response


def translate_logical_expression(expression):
    prompt = f"""EXAMPLES: 
         TASK: Convert (a|b) into natural english. RESPONSE: a or b should be true.
         TASK: Convert (a&b) into natural english. RESPONSE: a and b both should be true.
         TASK: Convert ((a&c)|(b&d)) into natural english. RESPONSE: both a and c should be true OR both b and d should be true.
         
         TASK: Convert the logical expression {expression} into natural English. Provide only the translation in a single clear sentence. RESPONSE:"""
    return generate(prompt, max_new_tokens=64)


import re

def replace_boolean_algebra_with_translation(text):
    # Regex pattern to match boolean algebra expressions
    pattern = r'\((?:[^()]*[&|][^()]*)+\)' 
    pattern2 = r'\((?:[a-zA-Z](?:[|&][a-zA-Z])*)\)'

    # Split the text into lines
    lines = text.split('\n')
    
    for i, line in enumerate(lines):
        # Find all boolean algebra expressions in the line
        matches = re.findall(pattern2, line)
        for match in matches:
            # Translate each expression and replace it in the line
            translation = translate_logical_expression(match)
            translation = "(" + translation + ")"
            line = line.replace(match, translation)
        lines[i] = line
    
    # Join the lines back into a single string
    return '\n'.join(lines)


Раскидываем текст по разным заголовкам

In [34]:
def structure_text_hmi(text):
    structured_text = {
        "preconditions": [],
        "main_scenario": [],
        "postconditions": []
    }

    current_section = None

    import string

    scenario_counter = 0
    for line in text.split("\n"):
        line = line.strip()
        if not line:
            continue
        from fuzzywuzzy import fuzz

        if fuzz.partial_ratio("Preconditions", line) > 95:
            current_section = "preconditions"
        elif fuzz.partial_ratio("Main Scenario", line) > 95:
            current_section = "main_scenario"
        elif fuzz.partial_ratio("Postconditions", line) > 95:
            current_section = "postconditions"
        elif fuzz.partial_ratio("Alternative Scenario", line) > 95:
            scenario_counter += 1
            current_section = f"alternative_scenario_{string.ascii_uppercase[scenario_counter - 1]}"
            structured_text[current_section] = []  # Start a new list for each alternative scenario
        elif current_section:
            structured_text[current_section].append(line.strip())

    return structured_text

def structure_text_to_dict_hmi(text_content):
    structured_text = structure_text_hmi(text_content)
    section_dict = {}

    
    for section, lines in structured_text.items():
        if section == "alternative_scenarios":
            section_text = []
            for scenario in lines:
                scenario_text = "\n".join(scenario)
                section_text.append(scenario_text)
            section_dict[section] = section_text
        else:
            section_text = "\n".join(lines)
            section_dict[section] = section_text

    return section_dict

Пример использования:

In [35]:
class Fact:
    def __init__(self, text, doc_name, section_name):
        self.text = text  # текст утверждения
        self.doc_name = doc_name  # название дока
        self.section_name = section_name  # название секции внутри которой находится факт
    
    def __repr__(self):
        return f"Text: {self.text}, DocName: {self.doc_name}, SectionName: {self.section_name}"


def process_docx_file_SSTS(file_path):
    docx_content = docx2python(file_path)
    text_content = docx_content.text
    first_line = text_content.split('\n', 1)[0]

    sections = parse_docx_text_SSTS(file_path)
    facts = []
    for section, content in sections.items():
        translated_content = content
        fact = Fact(text=translated_content, doc_name=first_line, section_name=section)
        facts.append(fact)
    return facts


def process_file_hmi(file_path):
    docx_content = docx2python(file_path)
    text_content = docx_content.text
    first_line = text_content.split('\n', 1)[0]

    sections_dict = structure_text_to_dict_hmi(text_content)
    facts = []
    
    for section, text in sections_dict.items():
        translated_text = text
        fact = Fact(text=translated_text, doc_name=first_line, section_name=section)
        facts.append(fact)
    
    return facts

In [36]:
def tell_about_fact(fact: Fact, is_rule=True):
    l = "A" if is_rule else "B"
    return f"Statement {l} is taken from document with name {fact.doc_name}, section with name {fact.section_name}. Statement {l} text: {fact.text}"

In [37]:
hmi_prefix = "/home/dev/case_data/train Атом/train data/HMI/UC-"
ssts_prefix = "/home/dev/case_data/train Атом/train data/SSTS/SSTS-"

doc_id = 28561

hmi_file = f"{hmi_prefix}{doc_id}.docx"
ssts_file = f"{ssts_prefix}{doc_id}.docx"

rules_data = process_file_hmi(hmi_file)
impl_data = process_docx_file_SSTS(ssts_file)

fewshot_examples = []

fewshot_examples.append([tell_about_fact(rules_data[0]), tell_about_fact(impl_data[0]), "No"])

fewshot_examples.append([tell_about_fact(rules_data[0]), tell_about_fact(impl_data[2]), "Yes"])


In [38]:
def build_prompt(factA: Fact, factB: Fact):
    system_prompt = "You are a compliance analyst tasked with verifying whether an implementation complies with a given regulation."
    user_prompt = f"""
        Logical statement A about car component:
        {factA}

        Logical statement B about car component:
        {factB}

        If you think that dismatch between A and B can lead to some unpleasant accidents, for example car crash, print "Yes"
        Otherwise print "No"

        Write only one word "Yes" or "No"
        Answer: 
    """
    return system_prompt, user_prompt


def build_prompt_description(factA: Fact, factB: Fact):
    system_prompt = "You are a compliance analyst tasked with verifying whether an implementation complies with a given regulation."
    user_prompt = f"""
        You will be provided with two statements:

        - Statement A (Regulation): Contains information from a regulation, policy, or specification outlining conditions and requirements.

        - Statement B (Implementation): Contains a description of an implementation that may or may not comply with the regulation specified in Statement A.


        Determine whether there are any discrepancies or inconsistencies between the regulation and the implementation.

        Also try to make your answer concise, write only the main diff and do not any other information

        Statement A text:
        {factA.text}

        Statement B text:
        {factB.text}

        Your concise discrepancy: 
    """
    return system_prompt, user_prompt

In [39]:
def get_facts(doc):
        content = docx2python(doc).text
        splitting_prompt = f"""You will see a document with list of rules and instructions. I need you to split into many little logical fragments. 
            Basically, I want you to give me a list of facts and list of keywords from a whole document. Output should be like:
            1) <fact 1>
            2) <fact 2>
            3) <fact 3>
            etc...
            Write numbers and facts and only them
            My document to split: {content}
        """
        res = [x for x in generate(splitting_prompt, max_new_tokens=1000).split("\n")]
        return res

def find_disrepancies(rule_doc, impl_doc, calc_facts=False):

    if calc_facts:
        impl_data = get_facts(impl_doc)
        rules_data = get_facts(rule_doc)
        
    else:
        rules_data = process_file_hmi(rule_doc)
        impl_data = process_docx_file_SSTS(impl_doc)

    print("RULES LIST")

    for rule in rules_data:
        print(rule)
    
    print("IMPL LIST")

    for impl in impl_data:
        print(impl)
    
    res = []
    for rule in rules_data:
        for impl in impl_data:
            system_prompt, user_prompt = build_prompt(rule, impl)
            ans = generate(user_prompt, max_new_tokens=1)
            print(ans.lower())
            if ans.lower() == "yes":
                system_prompt_descr, user_prompt_descr = build_prompt_description(rule, impl)
                descr = generate(user_prompt_descr, system_prompt_descr, max_new_tokens=100).strip().strip("\n")
                # system_prompt_severity, user_prompt_severity = build_prompt_severeness(descr)
                # severe = generate(user_prompt_severity, system_prompt_severity, max_new_tokens=1)

                res.append(
                    
                    impl
                )

    return res

In [40]:
# res = find_disrepancies(hmi_file, ssts_file, calc_facts=True)

KeyboardInterrupt: 

In [ ]:
res

In [22]:
# impl_data = get_facts(hmi_file)
# rules_data = get_facts(ssts_file)



In [ ]:
print(impl_data[:50])

In [ ]:
print(rules_data[:50])

In [ ]:
# Decode and print the response
response = generate(prompt)
print(response)

In [16]:
!pip install pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 62.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 KB 21.5 MB/s eta 0:00:00


In [51]:
import timeit
import pandas as pd
hmi_prefix = "/home/dev/case_data/test Атом/test data/HMI/UC-"
ssts_prefix = "/home/dev/case_data/test Атом/test data/SSTS/SSTS-"

doc_ids = [114671, 259571, 259572, 261611, 29448, 30364, 30365, 30370, 315231, 65831, 65832, 65833, 86921, 88001, 88002 ]

answer = {114671: "NC", 259571: "LC", 259572: "FC", 261611: "LC", 29448: "LC", 30364: "LC", 30365: "LC", 30370: "LC", 315231: "LC", 65831: "FC", 65832: "LC", 65833: "LC", 86921: "NC", 88001: "LC", 88002: "LC"}
df = pd.DataFrame(columns=['Number', 'Name', 'Differences', 'Description', 'Compliance Level'])
total_time = 0

for doc_id in doc_ids:
    
    hmi_file = f"{hmi_prefix}{doc_id}.docx"
    ssts_file = f"{ssts_prefix}{doc_id}.docx"
    rules_data = process_file_hmi(ssts_file)
    impl_data = process_docx_file_SSTS(hmi_file)
    prompt = f"""
    Evaluate the compliance of the document with the requirements using the rating system below:

    **Compliance Categories:**
    - FC (Fully Compliant): Perfect! Nothing can be improved.
    - LC (Largely Compliant): Generally correct. Some improvements may be needed (described in comments). No need for review.
    - PC (Partially Compliant): Major deviations. Improvements needed (described in comments). After improvement, review is required.
    - NC (Non-Compliant): Not compliant. Needs to be re-done and re-reviewed. Directions for update shown in comments.
    - NA (Not Applicable): Not applicable. Reason for non-applicability is described in comments.

    ### Instructions:
    1. Review the document content in "File" and compare it to the "REQUIREMENTS" provided.
    2. Based on this comparison, assign the appropriate two-letter compliance code (e.g., FC, LC, PC, NC, or NA).

    ### Notice (few-shot):

    LC is returned if the notifications for battery status are slightly delayed, if the sound alerts for seatbelt reminders are missing, if the user interface occasionally lags for less than 1 second, if some minor glitches occur in the navigation system.

    PC is returned if the regenerative braking system fails to engage properly at lower speeds, affecting braking consistency, if the navigation system misinterprets an address occasionally,

    NC is returned if the emergency braking system fails to activate when necessary, posing a direct safety risk, if the battery management system reports inaccurate charge levels, function can only be used when the vehicle is in a stopped state.

    NA is returned if the test involves an audio system, if IVI_IFT sends a video stream to SWP to notify the user

    ### Data:
    File: {impl_data[:50]}
    REQUIREMENTS: {rules_data[:50]}

    **Output Format**: Return ONLY ONE two-letter compliance code (e.g., FC, LC, PC, NC, or NA) as the final compliance rating. START YOUR ANSWER WITH THE CODE:
    """
    
    # Measure the time taken to generate the response
    start_time = timeit.default_timer()
    response = generate(prompt)
    elapsed_time = timeit.default_timer() - start_time
    total_time += elapsed_time
    print(f"Time taken for doc_id {doc_id}: {elapsed_time:.4f} seconds")

    # answer.append(response)
    # response = "NC"

    prompt2 = f"""
Output the differences in factual data in these files knowing their compliance level:

**Compliance Categories:**
- FC (Fully Compliant): Perfect! Nothing can be improved.
- LC (Largely Compliant): Generally correct. Some improvements may be needed (described in comments). No need for review.
- PC (Partially Compliant): Major deviations. Improvements needed (described in comments). After improvement, review is required.
- NC (Non-Compliant): Not compliant. Needs to be re-done and re-reviewed. Directions for update shown in comments.
- NA (Not Applicable): Not applicable. Reason for non-applicability is described in comments.

### Instructions:
1. Review the document content in "File" and compare it to the "REQUIREMENTS" provided.
2. Based on this comparison, assign the appropriate two-letter compliance code (e.g., FC, LC, PC, NC, or NA).

### Data:
Compliance level: {answer[doc_id]}
File1 (use-case): {impl_data[:150]}
File2 (requirements): {rules_data[:150]}

**Output Format**:Json with fields 
    'one_line_of_text_from_file_one_with_mismatch': ...,
    'one_line_of_text_from_file_two_with_mismatch': ...,
    'summary_of_mismatch': ...,
"""
    
    text = generate(prompt2, max_new_tokens=300)
    start_index_1 = text.find('"one_line_of_text_from_file_one_with_mismatch":') + len('"one_line_of_text_from_file_one_with_mismatch": ')
    end_index_1 = text.find('",', start_index_1)
    line1 = text[start_index_1+1:end_index_1]

    start_index_2 = text.find('"one_line_of_text_from_file_two_with_mismatch":') + len('"one_line_of_text_from_file_two_with_mismatch": ')
    end_index_2 = text.find('",', start_index_2)
    line2 = text[start_index_2+1:end_index_2]

    start_index_3 = text.find('"summary_of_mismatch":') + len('"summary_of_mismatch": ')
    end_index_3 = text.find('"\n', start_index_3)
    line3 = text[start_index_3+1:end_index_3]

    content = docx2python(ssts_file).text.split('\n', 1)[0]
    
    
    df = pd.concat([df, pd.DataFrame([{'Number': doc_id, 'Name': content, 'Differences': line3, 'Description': line1 + "\n" + line2, 'Compliance Level': response}])], ignore_index=True)


average_time = total_time / (len(doc_ids) + 0.000000001)
print(f"Average time taken per document: {average_time:.4f} seconds")




Time taken for doc_id 114671: 1.6420 seconds
Time taken for doc_id 259571: 1.3202 seconds
Time taken for doc_id 259572: 1.3211 seconds
Time taken for doc_id 261611: 1.3199 seconds
Time taken for doc_id 29448: 1.3198 seconds
Time taken for doc_id 30364: 1.3230 seconds
Time taken for doc_id 30365: 1.3216 seconds
Time taken for doc_id 30370: 1.3244 seconds
Time taken for doc_id 315231: 1.3237 seconds
Time taken for doc_id 65831: 1.3297 seconds
Time taken for doc_id 65832: 1.3286 seconds
Time taken for doc_id 65833: 1.3291 seconds
Time taken for doc_id 86921: 1.3279 seconds
Time taken for doc_id 88001: 1.3252 seconds
Time taken for doc_id 88002: 1.3270 seconds
Average time taken per document: 1.3455 seconds


In [52]:
df

,Number,Name,Differences,Description,Compliance Level
0,114671,Users can remotely control the DK deletion thr...,File1 does not contain any text related to rem...,\nUsers can remotely control the DK deletion t...,NA
1,259571,Mute or pause function,File1 is empty while File2 contains structured...,"\nDocName: Mute or pause function, SectionName...",NA
2,259572,Mute or pause function,File1 is empty while File2 contains detailed r...,"\nText: , DocName: Mute or pause function, Sec...",NA
3,261611,Automatic search,File1 is empty while File2 contains structured...,"\nText: , DocName: Automatic search, SectionNa...",NA
4,29448,Functional Description,"File1 is empty, while File2 contains sections ...","\nDocName: Functional Description, SectionName...",NA
5,30364,Maximum charging SOC value setting,"rmation, we need to identify any mismatches be...",ify any mismatches between `File1` and `File2`...,NA
6,30365,Functional Description,"File1 is empty, hence no direct line-by-line m...",\n,NA
7,30370,Functional Description,"File1 is empty, hence no direct line-by-line m...",\n,NA
8,315231,Favorite Song operation,File1 is empty while File2 contains structured...,"\nDocName: Favorite Song operation, SectionNam...",NA
9,65831,Make a call (B sample),"and instructions, let's analyze the differenc...","\n[{ \""Text\"": \""\",NA


In [53]:
df.to_csv('submission.csv', index=False)

In [14]:
print(answer)

['PC']
